# A demo of the `LinearOperators.jl` library

Solve the Laplace equation `Δu = f` over a `M × N` grid using finite differences. The numerical scheme reads in the real space

In [ ]:
using CairoMakie
using FFTW
using ImageFiltering
using LinearOperators

In [ ]:
const DIM = 2

In [ ]:
Lx, Ly = 1.0, 1.5

In [ ]:
Nx, Ny = 64, 128

In [ ]:
Δx, Δy = Lx / Nx, Ly / Ny

In [ ]:
nrows = ncols = Nx * Ny

In [ ]:
kx² = [(2 * sin(π * n / Nx))^2 for n = 0:(Nx - 1)]
ky² = [(2 * sin(π * n / Ny))^2 for n = 0:(Ny - 1)]

function mul_laplacian!(y, x, α, β)
    x_hat = fft(x)
    for nx = 1:Nx
        for ny = 1:Ny
            x_hat[nx, ny] *= kx²[nx] + ky²[ny]
        end
    end
    y .= α .* ifft(x_hat) .+ β .* y
end

In [ ]:
ux = [sin(2π * (n + 0.5) / Nx) for n = 0:(Nx-1)]
uy = [sin(4π * (n + 0.5) / Ny) for n = 0:(Ny-1)]
u = ux .* uy'

heatmap(u)

In [ ]:
Δu = Array{ComplexF64}(undef, Nx, Ny)
mul_laplacian!(Δu, u, 1.0, 0.0)
heatmap(real.(Δu))
#save("test.png", fig)

In [ ]:
w = centered(zeros(Float64, 3, 3))
w[-1, 0] = w[1, 0] = 1 / Δx^2
w[0, -1] = w[0, 1] = 1 / Δy^2
w[0, 0] = -2 * (1 / Δx^2 + 1 / Δy^2)
w

Δu_ref = imfilter(u, w, "circular", Algorithm.FIR())
heatmap(Δu_ref)

In [ ]:
Δu_ref ./ real(Δu)